# Exercício de avaliação 2 - Relatório/Notebook

### Enunciado


**2.** Use o “package” cryptography para \
    **a.** Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-128. \
    **b.** Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X25519 key exchange” e “Ed25519 Signing&Verification” para autenticação  dos agentes.  Deve incluir a confirmação da chave acordada. 

--------------------------------------------------------------------------------------------------------

### Introdução

A segurança da comunicação é essencial em aplicações modernas, especialmente quando há necessidade de privacidade e autenticidade na troca de mensagens entre agentes. Neste trabalho, implementamos um canal privado de comunicação assíncrona que utiliza técnicas de estruturas criptográficas avançadas abordadas nas aulas teóricas, estas irão nos garantir 3 objetivos fulcrais: **confidencialidade**, **integridade** e **autenticação** das mensagens trocadas.

A abordagem adotada baseia-se na utilização de **_Authenticated Encryption with Associated Data_** (AEAD) com **_Tweakable Block Ciphers_**, utilizando **_AES-128_** como cifra de bloco subjacente. A introdução a **_tweaks_** na cifra desta nova abordagem permite, em comparação com o exercício anterior, uma maior resistência a ataques, tornando o esquema mais robusto.

Para implementar um canal seguro entre os agentes, utilizamos o protocolo de acordo de chaves **_X25519_**, permitindo que as partes compartilhem uma chave secreta de forma segura (mesmo em canais inseguros). Além disso, implementamos **_Ed25519 Signing & Verification_** para autenticar os agentes.

O trabalho é desenvolvido utilizando a biblioteca `cryptography` do *Python*, que oferece suporte às primitivas criptográficas necessárias.

Para estruturar a implementação, tornar mais claro o que nos é pedido e garantir que o sistema cumpra os objetivos propostos, foi determinado um conjunto de requisitos funcionais, estes definem as funcionalidades essenciais que o sistema deve oferecer:

## Requisitos funcionais

**RF1** - O sistema deve implementar `Authenticated Encryption with Associated Data` (AEAD) baseada em `Tweakable Block Ciphers`, a cifra por blocos a usar é `AES-128`; \
**RF2** - O sistema deve utilizar `X25519 key exchange` para estabelecer uma chave secreta compartilhada entre os agentes. \
**RF3** - Os agentes devem ser autenticados através de `Ed25519 Signing & Verification`. \
**RF4** - O servidor deve validar a assinatura antes de processar qualquer mensagem recebida. \
**RF5** - Caso a confirmação falhe, a mensagem não deve ser aceite. \
**RF6** - O sistema deve utilizar uma fonte segura para geração de chaves e tweaks. \
**RF7** - As mensagens devem ser cifradas antes do envio. \
**RF8** - As mensagens devem ser decifradas ao serem recebidas. \
**RF9** - A implementação deve garantir confidencialidade, integridade e autenticidade das mensagens

Neste documento iremos mostrar como é que foram cumpridos todos os requisitos funcionais estabelecidos explicando pretinentemente o raciocínio por de trás da implementação.

------------------------------------------------------------------------------------------------------------------------------

## **RF1** - O sistema deve implementar `Authenticated Encryption with Associated Data` (AEAD) baseada em `Tweakable Block Ciphers`, a cifra por blocos a usar é `AES-128`; 

Como base (e com objetivos de simplicidade) foi implementado uma versão modificada da criptografia AES-128 no modo **ECB** (Electronic Codebook), incorporando um "_tweak_" na chave de cifragem. Este modo segue a estratégia demonstrada no diagrama:

![Modo ECB](ECB.png)

Fazemos uso do modo **ECB**, porém modificado, no sentido em que em vez de ser dado com _input_ o plaintext é dado a concatenação dos **dados associados** com **plaintext**, usamos como chave uma chave ajustada (**tweaked_key**) e é utilizado **AES-128** como a primtiva de cifração (PBC):


Fazendo $\kappa\;\equiv\; w\,\oplus\,k$, então obtemos a TPBC a partir da abordagem: $$\tilde{E}(w,k,x)\;\equiv\;E(\kappa,x)$$

Mais concretamente na prática:

In [ ]:
def tweakable_aes_encrypt(key: bytes, tweak: bytes, plaintext: bytes) -> bytes:
    """Aplica AES-128 no modo ECB com tweak XOR na chave."""
    assert len(key) == 16, "A chave deve ter 16 bytes (AES-128)."
    assert len(tweak) == 16, "O tweak deve ter 16 bytes."
    
    tweaked_key = bytes(a ^ b for a, b in zip(key, tweak))
    cipher = Cipher(algorithms.AES(tweaked_key), modes.ECB())
    encryptor = cipher.encryptor()

    # Adiciona padding caso a messagem não seja múltipla de 16 bytes
    padder = padding.PKCS7(128).padder() # 128 bits = 16 bytes * 8
    padded_plaintext = padder.update(plaintext) + padder.finalize()

    return encryptor.update(padded_plaintext) + encryptor.finalize()

(crypto_utils.py: Linhas 8-21)

O código definido tem 3 entradas:

- **_key_** (chave AES de longa duração de 16 bytes).
- **_tweak_** (Uma chave de curta duração de 16 bytes).
- **_plaintext_** (dados a serem criptografados).

No corpo da função (para além de definir pré-requisitos) começamos por operar um *XOR* entre a **_key_** e o **_tweak_**, gerando uma chave ajustada (**_tweaked_key_**). Para além disso aplicando uma estrutura criptográfica com AES-128-ECB:

- Usa `Cipher` da biblioteca cryptography para criar um cifrador AES-128 no modo ECB.
- Aplica **_PKCS7 padding_** para garantir que o *plaintext* tenha um tamanho múltiplo de 16 bytes.
- Cifra o resultado final e retorna o resultado.

Após definido o cifrador propriamente dito podemos agregar o processo total com:

In [2]:
def encrypt_with_aead(key, tweak, plaintext, associated_data):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(associated_data)
    associated_hash = digest.finalize()
    
    extended_plaintext = associated_hash + plaintext
    ciphertext = tweakable_aes_encrypt(key, tweak, extended_plaintext)
    
    return ciphertext

(crypto_utils.py: Linhas 39-47)

Este código aplica inicialmente um hash **_SHA256_** nos dados associados e concatenamos ambos para criar um input que vai oferecer robustez na cifra final. Por final podemos apenas processar um texto cifrado invocando a função anteriormente mencionada.

O uso do hash nas operações **_AEAD_** adiciona **integridade** e **autenticação** no processo de criptografia, ajudando a garantir que tanto os dados confidenciais quanto os dados associados sejam protegidos corretamente.

--------------------------------------------------------------------------------------------------------

## **RF2** - O sistema deve utilizar `X25519 key exchange` para estabelecer uma chave secreta compartilhada entre os agentes. 

Para obter uma comunicação segura foi nos proposto criar uma **chave pública** e uma **chave privada** fazendo uso da curva elíptica no protocolo Diffie-Hellman 25519. Para isso foi criada uma função `generate_x25519_keypair` que utiliza funções da biblioteca de `cryptography` do *Python* que facilita o processo.

Esta função gera um par de chaves X25519, primeiramente uma chave privada seguido da chave pública dependente da privada:

In [3]:
def generate_x25519_keypair():
    """Gera um par de chaves X25519."""
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key()
    return private_key, public_key

(crypto_utils.py: Linhas 64-68)

Dada a lógica de geração de chaves foi preciso também criar outra lógica para **troca de chaves** e **Derivação de chaves com HKDF**, dado que ao usar criptografia de curva elíptica como X25519, a chave privada nunca é compartilhada diretamente entre as partes. 

Em vez de partilhar diretamente as chaves privadas, ambas as partes trocam apenas as chaves públicas, e, por meio de um processo de troca segura, geram um **segredo partilhado**. Esse segredo é então transformado numa chave simétrica derviada através de uma HKDF, permitindo que a comunicação seja cifrada de forma segura sem revelar a chave privada:

In [4]:
def derive_shared_key(private_key, peer_public_key):
    """Deriva uma chave compartilhada usando X25519."""
    shared_secret = private_key.exchange(peer_public_key)
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=16,  # Chave de 16 bytes para AES-128
        salt=None,
        info=b'handshake data'
    ).derive(shared_secret)
    return derived_key

(crypto_utils.py: Linhas 76-85)

--------------------------------------------------------------------------------------------------------

## **RF3** - Os agentes devem ser autenticados através de `Ed25519 Signing & Verification`.
e
## **RF4** - O servidor deve validar a assinatura antes de processar qualquer mensagem recebida.

A autenticação de agentes utilizando `Ed25519 Signing & Verification` é essencial para garantir as características de **integridade** e **autenticidade** das mensagens trocadas entre os agentes no sistema. O `Ed25519` é um algoritmo de assinatura digital, como o X25519, na curva elíptica 25519. 

Ao assinar uma mensagem com a chave privada de um agente e verificar a assinatura com a chave pública, é possível se a mensagem não foi alterada e que realmente foi enviada pelo legítimo emissor. Esse processo previne falsificação de identidade e ataques de repetição, garantindo que apenas agentes autenticados possam interagir dentro do sistema de forma confiável.

Temos então uma geração de chaves para assinatura:

In [5]:
def generate_ed25519_keypair():
    """Gera um par de chaves Ed25519 para assinatura."""
    private_key = ed25519.Ed25519PrivateKey.generate()
    public_key = private_key.public_key()
    return private_key, public_key

(crypto_utils.py: Linhas 70-74)

E a lógica de assinatura e verificação com as chaves privadas e públicas respetivamente:

In [6]:
def sign_message(private_key, message: bytes) -> bytes:
    """Assina uma mensagem usando Ed25519."""
    return private_key.sign(message)

def verify_signature(public_key, message: bytes, signature: bytes):
    """Verifica a assinatura de uma mensagem com Ed25519."""
    try:
        public_key.verify(signature, message)
        return True
    except:
        return False

(crypto_utils.py: Linhas 87-97)

No caso do cliente este assina com a mensagem cifrada:

(Client.py: Linha 59)

E o servidor com a chave partilhada gerada:

(Server.py: Linha 42)

E ambos, para garantir a **integridade** e **autenticidade** das assinaturas verificam cada mensagem recebida:

--------------------------------------------------------------------------------------------------------

## **RF5** - Caso a confirmação falhe, a mensagem não deve ser aceite. 

Para evitar ataques de **Man-in-the-Middle** (um atacante poderia inserir uma chave falsa e interceptar ou modificar as mensagens) ou ataques de criação manual de mensagem decidimos ignorar todas as mensagens que não passam no teste de verificação.

Como mencionado anteriormente o cliente sempre que deteta uma falha na assinatura:

(Client.py: Linhas 41-43)

E o servidor rejeita a mensagem, neste caso, ignorando-a e iterando o ciclo para voltar a ficar à escuta de outras mensagens

(Server.py: Linhas 79-81)

---

## **RF6** - O sistema deve utilizar uma fonte segura para geração de chaves e tweaks.

## Geração de chaves

Para gerar os pares de chaves de troca, de assinaturas (e validação) e fazer a derivação para obter a chave final a ser usada utilizamos respetivamente:

(cripto_utils.py linhas 64-85)

## Geração de tweaks

Através da biblioteca `os` o cliente cria uma sequência de bytes (neste caso 16) pseudo-aleatória:

(Client.py: Linha 56)

E usa este tweak pseudo-aleatório (e outros argumentos) é usado para cifrar a mensagem:

(Client.py: Linha 58)

O servidor filtar o tweak da mensagem do cliente e utiliza-a para decifração:

(Servidor.py linhas 68-86)

Um novo tweak será definido pelo cliente antes de mandar outra mensagem, assim garantimos que cada mensagem é cifrada com uma chave diferente, o que evita **ataques de repetição** (ou seja, que mesmo que um atacante tenha acesso ao _ciphertext_ de uma mensagem anterior, ele não será capaz de decifrar uma nova mensagem com a mesma chave.), garantindo que a mesma chave de criptografia (`shared_key`) nunca seja usada com o mesmo **tweak** para múltiplas mensagens.

---

## **RF7** - As mensagens devem ser cifradas antes do envio.

(Cliente.py linhas 53-58)

---

## **RF8** - As mensagens devem ser decifradas ao serem recebidas.

(Servidor.py linhas 68-86)

---

## **RF9** - A implementação deve garantir confidencialidade, integridade e autenticidade das mensagens.

### Confidencialidade:

- AES-128 com tweak XOR na chave: A confidencialidade da mensagem é garantida pelo uso da cifra AES-128 no modo ECB com um "tweak" que modifica a chave de criptografia de maneira única para cada operação. Isso impede que a mesma chave seja reutilizada de maneira previsível, dificultando a criptografia de mensagens repetidas. O `tweakable_aes_encrypt` e `tweakable_aes_decrypt` implementam a cifração e decifração com esta técnica.

### Integridade:

- A função `encrypt_with_aead` utiliza uma técnica chamada **AEAD**. Ela calcula um hash **SHA-256** sobre os dados associados antes de criptografar os dados com **AES-128**, o que assegura que os dados associados não foram alterados (integridade). Ao decifrar, a função `decrypt_with_aead` voltar a calcular o hash **SHA-256** sobre os dados associados e compara com o hash armazenado. Se os hashes não coincidirem, um erro é levantado, indicando que os dados associados foram modificados.

### Autenticidade:

- Para garantir a autenticidade da mensagem, o código usa o algoritmo **Ed25519** de assinaturas digitais. A função `sign_message` assina a mensagem com a chave privada, e a função `verify_signature` permite verificar se a assinatura foi gerada pela chave privada associada à chave pública fornecida. Isso assegura que a mensagem veio de uma fonte legítima e que não foi modificada.

O uso do hash nas operações AEAD adiciona integridade e autenticação ao processo de criptografia, ajudando a garantir que tanto os dados confidenciais quanto os dados associados sejam protegidos corretamente.

-------

## Conclusão e observações

O grupo considera que foi capaz de implementar a **AEAD baseada em Tweakable Block Ciphers**, seguindo as diretrizes propostas. 

O **AES-128** foi empregado como a cifra base, com um tweak aleatório para cada mensagem, garantindo variação na chave de cifragem. Além disso, foi desenvolvido um canal privado de comunicação assíncrona, onde o acordo de chaves foi realizado com **X25519 key exchange**, e a autenticação dos agentes foi garantida pelo **Ed25519 Signing & Verification**, incluindo a confirmação da chave acordada. 

O grupo sentiu que as medidas tomatas asseguraram confidencialidade, autenticidade e integridade na comunicação.

Uma observação que o grupo quer apontar antes de dar por terminado o relatório está relacinado com implementação da cifra.
O grupo 7 decidiu utilizar o modo de cifra **ECB** que reconhecemos que tem inseguranças. Nomeadamente o **ECB** tem um problema de ser relativamente fácil detetar padrões, como todos os blocos são independentes de cada um (diferente de **CBC**).

Um exemplo clássico da demonstração desta vulnerabilidade está na encriptação de pixeis de imagens:

![Imagem Linux ECB](ECBLinux.png)

Mesmo cifrando os pixeis da imagem é simples de reconhecer um padrão mesmo que a informação objetivamente não seja a mesma.

O grupo decidiu manter o modo **ECB** por fins de simplicidade e queremos demonstrar (de uma maneira mais abstrata) que é relativamente fácil implementar outros tipos de PBC's.

Usando como referência a nossa função de encriptação:

In [7]:
def tweakable_aes_encrypt(key: bytes, tweak: bytes, plaintext: bytes) -> bytes:
    """Aplica AES-128 no modo ECB com tweak XOR na chave."""
    assert len(key) == 16, "A chave deve ter 16 bytes (AES-128)."
    assert len(tweak) == 16, "O tweak deve ter 16 bytes."
    
    tweaked_key = bytes(a ^ b for a, b in zip(key, tweak))
    cipher = Cipher(algorithms.AES(tweaked_key), modes.ECB())
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()

    return encryptor.update(padded_plaintext) + encryptor.finalize()

Poderíamos criar uma versão que utilizasse **CBC** (com especial atenção na utilização do initial vector):

In [8]:
def aes_cbc_encrypt(key: bytes, plaintext: bytes) -> bytes:
    iv = os.urandom(16)  # IV aleatório
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    padder = padding.PKCS7(128).padder() 
    padded_plaintext = padder.update(plaintext) + padder.finalize()

    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

    return iv + ciphertext  # Retorna IV + texto cifrado

Ou até mesmo com o modo **GCM** (que faz uso do nonce que poderia ser acordado pelo cliente e servidor):

In [9]:
def tweakable_aes_encrypt(key: bytes, nonce:bytes, tweak: bytes, plaintext: bytes, associated_data: bytes) -> bytes:
    """Aplica AES-128 no modo GCM com tweak XOR na chave."""
    assert len(key) == 16, "A chave deve ter 16 bytes (AES-128)."
    assert len(tweak) == 16, "O tweak deve ter 16 bytes."
    
    tweaked_key = bytes(a ^ b for a, b in zip(key, tweak))

    cipher = Cipher(algorithms.AES(tweaked_key), modes.GCM(nonce))
    encryptor = cipher.encryptor()

    encryptor.authenticate_additional_data(associated_data)

    padder = padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()

    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

    return nonce + ciphertext + encryptor.tag  # Retorna o nonce, o texto cifrado e a tag de autenticação

----

### **Ficheiro _Notebook_ do trabalho prático 1 - Exercício de avaliação alínea 2**
### Estruturas Criptográficas 2024/2025 

#### Realizado pelo grupo 7:
Paulo André Alegre Pinto PG55991 \
Pedro Miguel Dias Leiras PG55995